In [4]:
# set this to point to the parameter file used
const PARAMETER_FILE = "model.parameters.json"
# set this to point to the .results file containing the simulations
const RESULTS_FILE   = "experiments/Paper Run/Powder.results"

"experiments/Paper Run/Powder.results"

In [5]:
using JSON, SDDP,DataFrames, StatPlots
data = JSON.parsefile(PARAMETER_FILE)
results = SDDP.load(RESULTS_FILE);

In [6]:
using Plots
fntsm = Plots.font("times", 10.0pt)
fntlg = Plots.font("times", 12.0pt)
default(titlefont=fntlg, guidefont=fntlg, tickfont=fntsm, legendfont=fntsm,left_margin=10mm,bottom_margin=7.5mm)
default(size=(800,600),top_margin=0mm, right_margin=0mm)
gr()

Plots.GRBackend()

In [7]:
# Plot Figure 1 in the paper
plot(size=(750,500), left_margin=5mm, bottom_margin=5mm)
milking_requirements = data["energy_for_pregnancy"] + data["energy_for_bcs_milking"] + data["energy_for_maintenance"]
dry_requirements = data["energy_for_pregnancy"] + data["energy_for_bcs_dry"] + data["energy_for_maintenance"]

plot!(milking_requirements, label="Total: Milking",          linewidth=3, color="#00467F")
plot!(dry_requirements, label="Total: Dry",                  linewidth=3, color="#00467F", linestyle=:dot)
plot!(data["energy_for_bcs_milking"],label="BCS - Milking",  linewidth=3, color="#e65100")
plot!(data["energy_for_bcs_dry"],label="BCS - Dry",          linewidth=3, color="#e65100", linestyle=:dot)
hline!([data["energy_for_maintenance"]],label="Maintenance", linewidth=3, color="#009AC7", linestyle=:solid)
plot!(data["energy_for_pregnancy"], label="Pregnancy",       linewidth=3, color="#009AC7", linestyle=:dot)

plot!(ylims=(-100,900), xlabel="Weeks since calving", ylabel="Energy Requirement (MJ/Week)", legend=:topleft)
savefig("energy.pdf")

In [20]:
# Plot Figure 4
hgh = 3
plot(xlabel="Quantity (kg/Cow/Day)", ylabel="Price Multiplier",legend=false,ylims=(0,2hgh))
xbx = [0.0, 2.0, 2.0, 0.0]
ybx = [0, 0, 2hgh, 2hgh]
plot!(Plots.Shape(xbx, ybx), fillalpha=0.5, w=0, c="green", alpha=0)
plot!(Plots.Shape(2+xbx, ybx), fillalpha=0.5, w=0, c="orange", alpha=0)
plot!(Plots.Shape(4+xbx, ybx), fillalpha=0.5, w=0, c="red", alpha=0)
# plot!(Plots.Shape(6+xbx, ybx), fillalpha=0.66, w=0, c="red", alpha=0)
plot!([0, 3, 4, 6], 1+[0, 0, 1, 5], w=5, c="#00467F")
plot!(size=(4 * 175,3 * 175), left_margin=5mm, bottom_margin=5mm)
annotate!([(1,3,text("FEI Grade A")), (3,3,text("FEI Grade B")), (5,3,text("FEI Grade C"))])
savefig("fei_penalty.pdf")

In [19]:
# Plot Figure 5
df = readtable("data/TGA.daily.df.csv")
q = [0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]
rainfall = unstack(by(df, :week) do io
   DataFrame(
        rainfall = quantile(io[:rainfall], q),
        quantile = q
        ) 
        end, :quantile, :rainfall)

evapotranspiration = unstack(by(df, :week) do io
   DataFrame(
        evapotranspiration = quantile(io[:evapotranspiration], q),
        quantile = q
        ) 
        end, :quantile, :evapotranspiration)

plot(rainfall, Symbol(0.5), w=3, c="#00467F")
plot!(rainfall, Symbol(0.0), fill=(Symbol(1.0), "#00467F"), fillalpha=0.25, alpha=0)
plot!(rainfall, Symbol(0.1), fill=(Symbol(0.9), "#00467F"), fillalpha=0.25, alpha=0)
plot!(rainfall, Symbol(0.25), fill=(Symbol(0.75), "#00467F"), fillalpha=0.25, alpha=0)
rainfall_plot = plot!(legend=false, title="(a)", xlabel="Week of Year", ylabel="Rainfall\n(mm/Week)")

plot(evapotranspiration, Symbol(0.5), w=3, c="#00467F")
plot!(evapotranspiration, Symbol(0.0), fill=(Symbol(1.0), "#00467F"), fillalpha=0.25, alpha=0)
plot!(evapotranspiration, Symbol(0.1), fill=(Symbol(0.9), "#00467F"), fillalpha=0.25, alpha=0)
plot!(evapotranspiration, Symbol(0.25), fill=(Symbol(0.75), "#00467F"), fillalpha=0.25, alpha=0)
evapotranspiration_plot = plot!(legend=false, title="(b)", xlabel="Week of Year", ylabel="Evapotranspiration\n(mm/Week)")

plot(rainfall_plot, evapotranspiration_plot, layout=(1,2), size=(1000,375), left_margin=8mm, bottom_margin=5mm)
savefig("weather.pdf")

In [18]:
# Plot Figure 6

function getresults(results, key, idx, scalefactor=1.0)
    Q = [0.1, 0.25, 0.5, 0.75, 0.9]
    g2 = hcat([scalefactor*results[i][key] for i in 1:length(results) if results[i][:markov][40] == idx]...)
    g3 = hcat([quantile(g2[g,:], Q) for g in 1:size(g2, 1)]...)'
    DataFrame(x10=g3[:,1], x25=g3[:,2], x50=g3[:,3], x75=g3[:,4], x90=g3[:,5])
end

function plotpowder(results, key, ylims, ylabel,title,scalefactor=1.0)
    maxidx = indmax([results[i][:markov][52] for i in 1:length(results)])
    minidx = indmin([results[i][:markov][52] for i in 1:length(results)])
    blue = getresults(results, key, 1, scalefactor)
    grey = getresults(results, key, 2, scalefactor)
    orange = getresults(results, key, 3, scalefactor)

    plot()

    plot!(blue, :x10, fill=(:x90, "#00467F"), c="#00467F", fillalpha=0.25, alpha=1.0)
    plot!(grey, :x10, fill=(:x90, "grey"), c="grey", fillalpha=0.25, alpha=1)
    plot!(orange, :x10, fill=(:x90, "#e65100"), c="#e65100", fillalpha=0.25, alpha=1)
    plot!(blue, :x90, c="#00467F")
    plot!(grey, :x90, c="grey")
    plot!(orange, :x90, c="#e65100")

    plot!(blue, :x50, c="#00467F")
    plot!(grey, :x50, c="grey")
    plot!(orange, :x50, c="#e65100")
                    
    plot!(scalefactor*results[maxidx][key], color="#e65100", linewidth=3)
    plot!(scalefactor*results[minidx][key], color="#00467F", linewidth=3)
    xticks = collect(1:8.66:52)
    xticklabels = ["Aug", "Oct", "Dec", "Feb", "Apr", "Jun"]
    plot!(legend=false, ylims=ylims, ylabel=ylabel,title=title, xticks=(xticks, xticklabels), xlabel="")
end
       
# function plotpowder(results, key, ylims, ylabel,title,scalefactor=1.0)
#     maxidx = indmax([results[i][:markov][52] for i in 1:length(results)])
#     minidx = indmin([results[i][:markov][52] for i in 1:length(results)])
#     plot(
#         hcat([scalefactor*results[i][key] for i in 1:length(results) if results[i][:markov][40] == 2 && mod(i, 2) == 0]...),
#         color="gray", alpha=0.12, linewidth=1
#     )
#     plot!(
#         hcat([scalefactor*results[i][key] for i in 1:length(results) if results[i][:markov][40] == 1 && mod(i, 2) == 0]...),
#         color="#00467F", alpha=0.12, linewidth=1
#     )
    
#     plot!(
#         hcat([scalefactor*results[i][key] for i in 1:length(results) if results[i][:markov][40] == 3 && mod(i, 2) == 0]...),
#         color="#e65100", alpha=0.12, linewidth=1
#     )

#     plot!(scalefactor*results[maxidx][key], color="#e65100", linewidth=3)
#     plot!(scalefactor*results[minidx][key], color="#00467F", linewidth=3)
#     xticks = collect(1:8.66:52)
#     xticklabels = ["Aug", "Oct", "Dec", "Feb", "Apr", "Jun"]
#     plot!(legend=false, ylims=ylims, ylabel=ylabel,title=title, xticks=(xticks, xticklabels))
# end

function plotprice(results, title)
    maxidx = indmax([results[i][:markov][52] for i in 1:length(results)])
    minidx = indmin([results[i][:markov][52] for i in 1:length(results)])
    prices = Vector{Float64}[]
    for t in 1:24
        push!(prices, [6.0])
    end
    for t in 25:51
        push!(prices, [5.0, 6.0, 7.0])
    end
    push!(prices, [4.0, 5.0, 6.0, 7.0, 8.0])

    function toprices(markov)
       [prices[t][i] for (t,i) in enumerate(markov)] 
    end
    plot(
        hcat([toprices(results[i][:markov]) for i in 1:length(results) if results[i][:markov][40] == 2 && mod(i, 2) == 0]...),
        color="gray", linewidth=1, alpha=0.03
    )
    plot!(
        hcat([toprices(results[i][:markov]) for i in 1:length(results) if results[i][:markov][40] == 1 && mod(i, 2) == 0]...),
        color="#00467F", linewidth=1, alpha=0.03
    )

    plot!(
        hcat([toprices(results[i][:markov]) for i in 1:length(results) if results[i][:markov][40] == 3 && mod(i, 2) == 0]...),
        color="#e65100", linewidth=1, alpha=0.03
    )
    plot!(toprices(results[maxidx][:markov]), color="#e65100", linewidth=3)
    plot!(toprices(results[minidx][:markov]), color="#00467F", linewidth=3)
    xticks = collect(1:8.66:52)
    xticklabels = ["Aug", "Oct", "Dec", "Feb", "Apr", "Jun"]
    plot!(legend=false, ylabel="Forecast Price\n(\$/kg)",title=title, xticks=(xticks, xticklabels))
end

function objectiveplot(results, title)
    x1 = [r[:objective] - 3536.0 for r in results if r[:markov][40]==1]
    x2 = [r[:objective] - 3536.0 for r in results if r[:markov][40]==2]
    x3 = [r[:objective] - 3536.0 for r in results if r[:markov][40]==3]
    bins = -1500:200:6500
    y = zeros(size(bins, 1), 3)
    for (row, l, u) in zip(1:size(bins, 1), bins[1:end-1], bins[2:end])
        y[row, 1] = sum(l .<= x1 .<= u)
        y[row, 2] = sum(l .<= x2 .<= u)
        y[row, 3] = sum(l .<= x3 .<= u)
    end
    groupedbar(y, bar_position = :stack, bar_width=1, c=["#00467F" "gray" "#e65100"], fillalpha=0.75)

                                                                                                    
    maxidx = indmax([results[i][:markov][52] for i in 1:length(results)])
    minidx = indmin([results[i][:markov][52] for i in 1:length(results)])
    vline!([indmin(abs.(bins - (results[maxidx][:objective] - 3536.0)))], color="#e65100", linewidth=3)
    vline!([indmin(abs.(bins - (results[minidx][:objective] - 3536.0)))], color="#00467F", linewidth=3)
    plot!(xticks=(3:10:size(y, 1), 0.5 * (bins[1:end-1] + bins[2:end])[3:10:end]))
    plot!(ylabel="Number of Simulations", xlabel="Operating Profit (\$/Ha)\n", legend=false)
    plot!(title=title)                                                                                            
end

plot(
    objectiveplot(results, "(a)"),
    plotprice(results, "(b)"),
    plotpowder(results, :C, (0, 3), "Cows Milking\n(Cows/Ha)", "(c)"),
    plotpowder(results, :P, (1.5,3), "Pasture Cover\n(t/Ha)","(d)", 0.001),
    plotpowder(results, :W, (0,150), "Soil Moisture\n(mm)","(e)"),
#     plotpowder(results, :ev, (0,45), "Evapotranspiration\n(mm/Week)", "(e)"),
    plotpowder(results, :gr, (0,70), "Pasture Growth\n(kg/Day)", "(f)"),
    plotpowder(results, :fₛ, (0,150), "Supplement Fed\n(kg/Week)", "(g)"),
    plotpowder(results, :milk, (0,2.5), "Milk Production\n(kgMS/Cow/Day)", "(h)", 1 / 3 / 7),
    layout=(4,2), size=(1000,1200)
)
savefig("farm.pdf")